In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Langkah 1: Tentukan Model Pydantic untuk Output Berstruktur

Model-model ini menentukan **skema** yang akan dikembalikan oleh ejen. Menggunakan `response_format` dengan Pydantic memastikan:
- ✅ Ekstraksi data yang selamat jenis
- ✅ Pengesahan automatik
- ✅ Tiada ralat parsing daripada respons teks bebas
- ✅ Penghalaan bersyarat yang mudah berdasarkan medan


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Langkah 2: Cipta Alat Tempahan Hotel

Alat ini akan digunakan oleh **availability_agent** untuk memeriksa sama ada bilik tersedia. Kami menggunakan penghias `@ai_function` untuk:
- Menukar fungsi Python menjadi alat yang boleh dipanggil oleh AI
- Menjana skema JSON secara automatik untuk LLM
- Mengendalikan pengesahan parameter
- Membolehkan pemanggilan automatik oleh ejen

Untuk demo ini:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → Ada bilik ✅
- **Semua bandar lain** → Tiada bilik ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Langkah 3: Tentukan Fungsi Keadaan untuk Penghalaan

Fungsi-fungsi ini memeriksa respons agen dan menentukan laluan mana yang perlu diambil dalam aliran kerja.

**Corak Utama:**
1. Periksa jika mesej adalah `AgentExecutorResponse`
2. Huraikan output berstruktur (model Pydantic)
3. Kembalikan `True` atau `False` untuk mengawal penghalaan

Aliran kerja akan menilai keadaan-keadaan ini pada **tepi** untuk menentukan pelaksana mana yang akan dipanggil seterusnya.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Langkah 4: Cipta Executor Paparan Tersuai

Executor adalah komponen aliran kerja yang melaksanakan transformasi atau kesan sampingan. Kita menggunakan dekorator `@executor` untuk mencipta executor tersuai yang memaparkan hasil akhir.

**Konsep Utama:**
- `@executor(id="...")` - Mendaftarkan fungsi sebagai executor aliran kerja
- `WorkflowContext[Never, str]` - Petunjuk jenis untuk input/output
- `ctx.yield_output(...)` - Menghasilkan keputusan akhir aliran kerja


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Langkah 5: Muatkan Pembolehubah Persekitaran

Konfigurasikan klien LLM. Contoh ini berfungsi dengan:
- **Model GitHub** (Tahap percuma dengan token GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Langkah 6: Cipta Ejen AI dengan Output Berstruktur

Kami mencipta **tiga ejen khusus**, setiap satu dibungkus dalam `AgentExecutor`:

1. **availability_agent** - Memeriksa ketersediaan hotel menggunakan alat
2. **alternative_agent** - Mencadangkan bandar alternatif (jika tiada bilik)
3. **booking_agent** - Menggalakkan tempahan (jika bilik tersedia)

**Ciri Utama:**
- `tools=[hotel_booking]` - Menyediakan alat kepada ejen
- `response_format=PydanticModel` - Memaksa output JSON berstruktur
- `AgentExecutor(..., id="...")` - Membungkus ejen untuk kegunaan aliran kerja


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Langkah 7: Bina Aliran Kerja dengan Tepi Bersyarat

Sekarang kita gunakan `WorkflowBuilder` untuk membina graf dengan penghalaan bersyarat:

**Struktur Aliran Kerja:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Kaedah Utama:**
- `.set_start_executor(...)` - Menetapkan titik permulaan
- `.add_edge(from, to, condition=...)` - Menambah tepi bersyarat
- `.build()` - Menyelesaikan aliran kerja


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Langkah 8: Jalankan Kes Ujian 1 - Bandar TANPA Ketersediaan (Paris)

Mari kita uji laluan **tanpa ketersediaan** dengan meminta hotel di Paris (yang tiada bilik dalam simulasi kita).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Langkah 9: Jalankan Kes Ujian 2 - Bandar DENGAN Ketersediaan (Stockholm)

Sekarang mari kita uji laluan **ketersediaan** dengan meminta hotel di Stockholm (yang mempunyai bilik dalam simulasi kita).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Perkara Penting dan Langkah Seterusnya

### ✅ Apa Yang Anda Telah Pelajari:

1. **Corak WorkflowBuilder**
   - Gunakan `.set_start_executor()` untuk menentukan titik permulaan
   - Gunakan `.add_edge(from, to, condition=...)` untuk penghalaan bersyarat
   - Panggil `.build()` untuk memuktamadkan aliran kerja

2. **Penghalaan Bersyarat**
   - Fungsi syarat memeriksa `AgentExecutorResponse`
   - Analisis output berstruktur untuk membuat keputusan penghalaan
   - Kembalikan `True` untuk mengaktifkan laluan, `False` untuk melangkauinya

3. **Integrasi Alat**
   - Gunakan `@ai_function` untuk menukar fungsi Python kepada alat AI
   - Ejen memanggil alat secara automatik apabila diperlukan
   - Alat mengembalikan JSON yang boleh dianalisis oleh ejen

4. **Output Berstruktur**
   - Gunakan model Pydantic untuk pengekstrakan data yang selamat jenis
   - Tetapkan `response_format=MyModel` semasa mencipta ejen
   - Analisis respons dengan `Model.model_validate_json()`

5. **Executor Tersuai**
   - Gunakan `@executor(id="...")` untuk mencipta komponen aliran kerja
   - Executor boleh mengubah data atau melaksanakan kesan sampingan
   - Gunakan `ctx.yield_output()` untuk menghasilkan hasil aliran kerja

### 🚀 Aplikasi Dunia Sebenar:

- **Tempahan Perjalanan**: Periksa ketersediaan, cadangkan alternatif, bandingkan pilihan
- **Khidmat Pelanggan**: Penghalaan berdasarkan jenis isu, sentimen, keutamaan
- **E-dagang**: Periksa inventori, cadangkan alternatif, proses pesanan
- **Moderasi Kandungan**: Penghalaan berdasarkan skor ketoksikan, bendera pengguna
- **Aliran Kerja Kelulusan**: Penghalaan berdasarkan jumlah, peranan pengguna, tahap risiko
- **Pemprosesan Berbilang Peringkat**: Penghalaan berdasarkan kualiti data, kelengkapan

### 📚 Langkah Seterusnya:

- Tambahkan syarat yang lebih kompleks (kriteria berganda)
- Laksanakan gelung dengan pengurusan keadaan aliran kerja
- Tambahkan sub-aliran kerja untuk komponen yang boleh digunakan semula
- Integrasi dengan API sebenar (tempahan hotel, sistem inventori)
- Tambahkan pengendalian ralat dan laluan sandaran
- Visualisasikan aliran kerja dengan alat visualisasi terbina dalam



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
